In [ ]:
#### ANI AEV Descriptors ##########
#### Use with ace environement ####


# import numpy as np
# import torch
# import torchani
# from ase import Atoms
# from ase.io import read, iread

# aev_computer = torchani.AEVComputer.cover_linearly(
#     radial_cutoff=5.2,  # Cutoff for radial part (Angstrom) --> original 5.2
#     angular_cutoff=3.5,  # Cutoff for angular part (Angstrom) --> original 3.5
#     radial_eta=16.0,  # Radial resolution
#     angular_eta=8.0,  # Shifted radial grid
#     radial_dist_divisions=16,  # Angular resolution
#     angular_dist_divisions=4,  # Angular scaling factor
#     zeta=32.0,  # Shifted angular grid
#     angle_sections=8,  # Shifted angular angles
#     num_species=4,
# )

# SC = torchani.SpeciesConverter(["H", "C", "N", "O"])

# mol_desc = []

# for split in ["val", "train", "test"]:
#     all_des = []
#     for index, molecule in enumerate(iread(f"{split}.xyz")):

#         atomicnums = torch.tensor(molecule.get_atomic_numbers())
#         positions = molecule.get_positions()

#         sc = SC((atomicnums, positions))
#         species = torch.tensor([sc.species.tolist()])  # Convert to ANI format

#         coordinates = torch.tensor([positions], dtype=torch.float32)

#         aev_output = aev_computer((species, coordinates))
#         mol_desc.append(aev_output.aevs)  # Shape: (num_atoms, AEV_dim)

#         if len(mol_desc) == 3:
#             all_des.append(
#                 np.concatenate(
#                     (
#                         mol_desc[0].numpy()[0],
#                         mol_desc[1].numpy()[0],
#                         mol_desc[2].numpy()[0],
#                     ),
#                     axis=1,
#                 )
#             )  # Must be of shape: (1, num_atoms, 1152)
#             mol_desc = []

#     np.savez(f"{split}_aniAEV_r_ts_p.npz", *all_des)

In [ ]:
##### Create SOAP descriptors for reactants, TS and products #####

# species = ["H", "C", "O", "N"]
# r_cut = 6.0
# n_max = 8
# l_max = 6

# # Setting up the SOAP descriptor
# soap = SOAP(
#     species=species,
#     periodic=False,
#     r_cut=r_cut,
#     n_max=n_max,
#     l_max=l_max,
# )

# path = "data/rdb7/ground_truth"

# for split in ["train", "test", "val"]:

#     df = pd.read_csv(f"{path}/{split}.csv")
#     xyz_data = read(f"{path}/{split}.xyz", ":")

#     all_desc = []
#     xyz_index = 0

#     for index, row in tqdm.tqdm(df.iterrows()):

#         smiles = df.loc[index]["smiles"].split(">>")[0]

#         params = Chem.SmilesParserParams()
#         params.removeHs = False
#         mol = Chem.MolFromSmiles(smiles, params)

#         atom_map_numbers = np.array([a.GetAtomMapNum() for a in mol.GetAtoms()])

#         atom_symbols = xyz_data[xyz_index].get_chemical_symbols()
#         sorted_atom_symbols = [
#             atom_symbols[i - 1] for i in atom_map_numbers
#         ]  # Sort atom symbols according to atom mapping numbers

#         positions = xyz_data[xyz_index].get_positions()
#         sorted_positions = [
#             positions[i - 1] for i in atom_map_numbers
#         ]  # Sort positions according to atom mapping numbers

#         ### Positions for TS #####
#         xyz_index += 1
#         positions_ts = xyz_data[xyz_index].get_positions()
#         sorted_positions_ts = [positions_ts[i - 1] for i in atom_map_numbers]

#         ### Positions for Product #####
#         xyz_index += 1
#         positions_prod = xyz_data[xyz_index].get_positions()
#         sorted_positions_prod = [positions_prod[i - 1] for i in atom_map_numbers]
#         xyz_index += 1

#         molecule = Atoms(positions=sorted_positions, symbols=sorted_atom_symbols)
#         molecule_ts = Atoms(positions=sorted_positions_ts, symbols=sorted_atom_symbols)
#         molecule_prod = Atoms(
#             positions=sorted_positions_prod, symbols=sorted_atom_symbols
#         )

#         desc = soap.create(molecule, n_jobs=8)
#         desc_ts = soap.create(molecule_ts, n_jobs=8)
#         desc_prod = soap.create(molecule_prod, n_jobs=8)

#         all_desc.append(np.hstack([desc, desc_ts, desc_prod]))
#         # all_desc.append(desc_ts)

#     np.savez_compressed(f"{path}/{split}_soap_r_ts_p.npz", *all_desc)


In [ ]:
# ### Equiformer Descriptors --> Working only with the fairchem environment!!

# from fairchem.core import OCPCalculator
# from ase.io import iread
# from fairchem.core.datasets import data_list_collater
# import torch
# from tqdm.auto import tqdm
# import pandas as pd
# import numpy as np
# from rdkit import Chem

# calc = OCPCalculator(
#     checkpoint_path="eqV2_31M_omat.pt",
#     cpu=False,
#     seed=0,
# )

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# params = Chem.SmilesParserParams()
# params.removeHs = False


# split = "train"
# extxyz_file = f'{split}.xyz'
# training_data = []

# #equiformer specific disregard
# for a, atoms in tqdm(enumerate(iread(extxyz_file))):
#     data_object = calc.a2g.convert(atoms)
#     batch = data_list_collater([data_object], otf_graph=True)
#     batch = batch.to("cuda:0")
#     with torch.no_grad():
#         training_data.append(
#             calc.trainer.model.backbone.forward(batch)["node_embedding"]
#             .embedding.narrow(1, 0, 1)
#             .detach()
#         )
#     batch = batch.to("cpu")
#     del batch, data_object
#     torch.cuda.empty_cache()

# d = pd.read_csv(f"{split}.csv")
# V_fs = []
# for i in range(len(d)):
#     rsmi = d['smiles'][i].split(">")[0]
#     r = Chem.MolFromSmiles(rsmi, params)
#     ridx = np.array([a.GetAtomMapNum()-1 for a in r.GetAtoms()])

#     V_f = np.concatenate((training_data[3*i+0][ridx,0,:].cpu(),training_data[3*i+1][ridx,0,:].cpu(),training_data[3*i+2][ridx,0,:].cpu()),axis=1)
#     V_fs.append(V_f)
# np.savez(f"{split}_eqV2_r_ts_p.npz", *V_fs)